## Use this notebook to clean demographic data 

In [1]:
import pandas as pd
import numpy as np

In [489]:
cen = pd.read_csv("cc-est2021-alldata-53.csv")
income = pd.read_csv("median_household_income_estimates.csv")

Source URL: https://www.ofm.wa.gov/washington-data-research/economy-and-labor-force/median-household-income-estimates


- Money income, as defined by the Bureau of the Census, includes wage or salary income, self-employment income, interest, dividend, rental income, social security or other public assistance income, retirement, and disability income; etc.		

---
Source URL: https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2020-2021/

- The estimates are developed from a base that incorporates the 2020 Census, Vintage 2020 estimates, and 2020 Demographic Analysis estimates

In [490]:
income.rename(columns={'Unnamed: 0':'County','Unnamed: 1':'medinc19','Unnamed: 2':'estinc2020','Unnamed: 3':'estinc2021'}, inplace=True)

income.drop(index=income.index[:4],inplace=True)
income.drop(income.iloc[:, 4:37], axis=1, inplace = True)

In [491]:
income.dropna(axis=0, inplace=True)

In [492]:
income[['medinc19','estinc2020','estinc2021']] = income[['medinc19','estinc2020','estinc2021']].replace(',', '', regex=True).astype(int)

In [156]:
# counties we want

counties = ['Franklin', 'Pierce', 'Grant', 'Yakima', 'King', 'Snohomish', 'Spokane', 'Mason']

income = income.loc[income['County'].isin(counties)]

In [160]:
income.to_csv("med_income_washco.csv")

In [427]:
cen['CTYNAME'] = cen['CTYNAME'].replace(r'(\s*)County', '', regex=True)

In [447]:
# counties we want

counties = ['Franklin', 'Pierce', 'Grant', 'Yakima', 'King', 'Snohomish', 'Spokane', 'Mason']

target = cen.loc[cen['CTYNAME'].isin(counties)]

In [448]:
# year = 4/1/2020

target = target.loc[target['YEAR']==2] 

In [90]:
colname = ['NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE','NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE', 'H_MALE', 'H_FEMALE', 'NHTOM_MALE', 'NHTOM_FEMALE']
sumlist = target[colname].sum(axis=1)

In [449]:
target.drop(target.iloc[:, 10:34], axis=1, inplace=True)

In [450]:
target.drop(target.iloc[:, 22:32], axis=1, inplace=True)

In [451]:
target.drop(target.iloc[:, 24:46], axis=1, inplace=True)

In [131]:
colname = ['NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE','NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE', 'H_MALE', 'H_FEMALE', 'NHTOM_MALE', 'NHTOM_FEMALE']
sumlist = target[colname].sum(axis=1)

In [452]:
target['totalWA'] = target[['NHWA_MALE', 'NHWA_FEMALE']].sum(axis=1)
target['totalBA'] = target[['NHBA_MALE', 'NHBA_FEMALE']].sum(axis=1)
target['totalIA'] = target[['NHIA_MALE', 'NHIA_FEMALE']].sum(axis=1)
target['totalAA'] = target[['NHAA_MALE', 'NHAA_FEMALE']].sum(axis=1)
target['totalNA'] = target[['NHNA_MALE', 'NHNA_FEMALE']].sum(axis=1)
target['totalH'] = target[['H_MALE', 'H_FEMALE']].sum(axis=1)
target['totalTOM'] = target[['NHTOM_MALE', 'NHTOM_FEMALE']].sum(axis=1)

In [453]:
target.drop(['NHWA_MALE', 'NHWA_FEMALE', 'NHBA_MALE', 'NHBA_FEMALE', 'NHIA_MALE','NHIA_FEMALE', 'NHAA_MALE', 'NHAA_FEMALE', 'NHNA_MALE', 'NHNA_FEMALE', 'H_MALE', 'H_FEMALE', 'NHTOM_MALE', 'NHTOM_FEMALE'], axis=1, inplace=True)

In [143]:
target.to_csv("highriskdemographics.csv")

### Pre-Processing Main Data

In [205]:
# start from here

data = pd.read_csv("crashdata_updated.csv", low_memory=False) # some cols have mixed dtypes...
data["crash_dt"]= pd.to_datetime(data["crash_dt"])
data["licstate"]= data["licstate"].astype(str)

# Among drivers involved in fatal crashes, what proportion are involved in crashes in communities where they live?

# How do we want to define "community"? Zip code should be fine... 
data['dzip'] = data['dzip'].astype(str)
# get rid of the trailing zeros
data['dzip'] = data['dzip'].replace(r'\.0$', '', regex=True)

In [206]:
# In order for this calculation to be accurate, we need to remove the rows where BOTH zip code and state is "unknown" or "not reported" because the driver zip for each of these rows won't match the crash zip but we have no way validating where the driver is ACTUALLY from-- we cannot assume that the driver is from out of state/country 

# Answers hinge on the assumption that the driver's zip code for each observation is accurate 
for index, row in data.iterrows():
    if row['dzip']=='99999' and row['licstate']=='99':
        data.drop(index, inplace=True)
    if row['dzip']=='99999' and row['licstate']=='98':
        data.drop(index, inplace=True)
    if row['dzip']=='99998' and row['licstate']=='99':
        data.drop(index, inplace=True)
    if row['dzip']=='99998' and row['licstate']=='98':
        data.drop(index, inplace=True)
    if row['dzip']=='99998' and row['licstate']=='53':
        data.drop(index, inplace=True)

In [207]:
# should be done

data.reset_index(inplace=True)

In [208]:
# Source: https://github.com/scpike/us-state-county-zip
citydata = pd.read_csv("geo-data.csv")

giszips = pd.read_csv("zip_usa_0.csv")

In [211]:
giszips['ZIP_CODE'] = giszips['ZIP_CODE'].astype(str)

In [214]:
# match city to zip codes

def matchcityGIS(df, dct):
    dct = dict(zip(giszips['ZIP_CODE'], giszips['PO_NAME']))
    df['dcity']=df['dzip'].map(dct).fillna('00') # map over df 
    df['crashcity']=df['crashzip'].map(dct).fillna('00') # map over df 
    return df

In [215]:
# GIS ZIPS
GISZIP = dict(zip(giszips['ZIP_CODE'], giszips['PO_NAME']))
matchcityGIS(data, GISZIP)

,index,Unnamed: 0,year,case,par,repjur,crash_dt,crash_tm,accday,accmon,...,crf1,crf2,crf3,CoRoadName,CoMP,IntCoRoadName,IntCoMP,crashzip,dcity,crashcity
0,0,0,2017,1,E628946,2.0,2017-01-01,2:12,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98201,Everett,Everett
1,1,1,2017,2,E627989,26.0,2017-01-02,17:14,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98359,Tacoma,Olalla
2,2,2,2017,4,3747633,263.0,2017-01-01,18:47,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98103,Seattle,Seattle
3,3,3,2017,5,E628691,4.0,2017-01-01,3:50,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99224,Mead,Spokane
4,4,4,2017,6,3746306,263.0,2017-01-05,9:53,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98104,Snoqualmie,Seattle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4026,4127,4127,2021,636,Incid08,334.0,2021-08-08,9:38,8,8,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,99116,Nespelem,Coulee Dam
4027,4128,4128,2021,638,Incid09,334.0,2021-09-05,1:36,5,9,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,99138,Inchelium,Inchelium
4028,4129,4129,2021,639,EB48605,263.0,2021-04-21,17:32,21,4,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,98144,Lacey,Seattle
4029,4130,4130,2021,639,EB48605,263.0,2021-04-21,17:32,21,4,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,98144,Seattle,Seattle


In [229]:
# impute missing cities

data.loc[data.crashzip=='V4L','crashcity'] = "Seattle"
data.loc[data.crashzip=='99107','crashcity'] = "Boyds"

data.loc[data.dzip=='0','dcity'] = "Canada"
data.loc[data.dzip=='98205','dcity'] = "Everett"
data.loc[data.dzip=='98455','dcity'] = "Tacoma"
data.loc[data.dzip=='99327','dcity'] = "Othello"
data.loc[data.dzip=='98132','dcity'] = "Seattle"

data.loc[data.licstate=='11','dcity'] = "District of Columbia"
data.loc[data.licstate=='95','dcity'] = "Canada"
data.loc[data.licstate=='1','dcity'] = "Alabama"

In [239]:
data.to_csv("crashdata_updated.csv")

## Add population size to zips in riskrate

In [356]:
riskrate = pd.read_csv("riskrate.csv")
riskrate['zipcode'] = riskrate['zipcode'].astype(str)

In [357]:
POPZIP = dict(zip(giszips['ZIP_CODE'], giszips['POP2021']))
riskrate['zip_pop']=riskrate['zipcode'].map(POPZIP).fillna('00') # map over df

In [358]:
SQMIZIP = dict(zip(giszips['ZIP_CODE'], giszips['SQMI']))
riskrate['SQMI']=riskrate['zipcode'].map(SQMIZIP).fillna('00') # map over df

In [359]:
CITYZIP = dict(zip(giszips['ZIP_CODE'], giszips['PO_NAME']))
riskrate['city']=riskrate['zipcode'].map(CITYZIP).fillna('00') # map over df

In [360]:
COUNTYZIP = dict(zip(citydata['zipcode'], citydata['county']))
riskrate['county']=riskrate['zipcode'].map(COUNTYZIP).fillna('00') # map over df

In [361]:
riskrate.loc[riskrate.city=='Bonney Lake','county'] = "Pierce"

In [362]:
COUNTYPOPZIP = dict(zip(countypops['CTYNAME'], countypops['TOT_POP']))
riskrate['county_pop']=riskrate['county'].map(COUNTYPOPZIP).fillna('00') # map over df

In [363]:
riskrate[['zip_pop','county_pop']] = riskrate[['zip_pop','county_pop']].replace(r'\.0$', '', regex=True).astype(int)
riskrate["percent_pop"] = riskrate["zip_pop"]/riskrate["county_pop"]

In [364]:
MaleZIP = dict(zip(countypops['CTYNAME'], countypops['TOT_MALE']))
riskrate['male_pop']=riskrate['county'].map(MaleZIP).fillna('00')

FemZIP = dict(zip(countypops['CTYNAME'], countypops['TOT_FEMALE']))
riskrate['fem_pop']=riskrate['county'].map(FemZIP).fillna('00')

In [365]:
waZIP = dict(zip(countypops['CTYNAME'], countypops['totalWA']))
riskrate['wa_pop']=riskrate['county'].map(waZIP).fillna('00')

baZIP = dict(zip(countypops['CTYNAME'], countypops['totalBA']))
riskrate['ba_pop']=riskrate['county'].map(baZIP).fillna('00')

iaZIP = dict(zip(countypops['CTYNAME'], countypops['totalIA']))
riskrate['ia_pop']=riskrate['county'].map(iaZIP).fillna('00')

aaZIP = dict(zip(countypops['CTYNAME'], countypops['totalAA']))
riskrate['aa_pop']=riskrate['county'].map(aaZIP).fillna('00')

naZIP = dict(zip(countypops['CTYNAME'], countypops['totalNA']))
riskrate['na_pop']=riskrate['county'].map(naZIP).fillna('00')

HZIP = dict(zip(countypops['CTYNAME'], countypops['totalH']))
riskrate['h_pop']=riskrate['county'].map(HZIP).fillna('00')

tomZIP = dict(zip(countypops['CTYNAME'], countypops['totalTOM']))
riskrate['tom_pop']=riskrate['county'].map(tomZIP).fillna('00')

In [366]:
riskrate[['wa_pop','ba_pop','ia_pop','aa_pop','na_pop','h_pop','tom_pop']] = riskrate[['wa_pop','ba_pop','ia_pop','aa_pop','na_pop','h_pop','tom_pop']].replace(r'\.0$', '', regex=True).astype(int)

In [373]:
riskrate.to_csv("riskrate.csv")

In [369]:
riskrate.loc[riskrate.county=='00','percent_pop'] = 0

In [296]:
density = pd.read_csv("popden_city.csv")
ctypop = pd.read_csv("highriskdemographics.csv")

In [282]:
density.dropna(axis=0, inplace=True)

In [283]:
density.drop(density.iloc[:,2:22],axis=1, inplace = True)

In [292]:
density.drop(density.iloc[:,8:17],axis=1, inplace=True)

In [ ]:
ageZIP = dict(zip(countypops['CTYNAME'], countypops['totalTOM']))
riskrate['tom_pop']=riskrate['county'].map(ageZIP).fillna('00')

In [378]:
ctypop.loc[ctypop.AGEGRP==4]

,Unnamed: 0,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,totalWA,totalBA,totalIA,totalAA,totalNA,totalH,totalTOM
4,593,50,53,21,Washington,Franklin,2,4,8045,4047,3998,2094,161,29,141,10,5446,164
23,707,50,53,25,Washington,Grant,2,4,7502,3891,3611,2913,76,59,44,14,4213,183
42,935,50,53,33,Washington,King,2,4,122183,61596,60587,58700,9468,770,23209,1435,18387,10214
61,1277,50,53,45,Washington,Mason,2,4,3414,1828,1586,2285,48,144,34,44,633,226
80,1505,50,53,53,Washington,Pierce,2,4,55229,28519,26710,29875,3907,725,3599,1226,9756,6141
99,1733,50,53,61,Washington,Snohomish,2,4,47313,24393,22920,27038,1719,592,5360,322,8621,3661
118,1790,50,53,63,Washington,Spokane,2,4,34090,17480,16610,26375,737,572,869,309,3003,2225
137,2189,50,53,77,Washington,Yakima,2,4,19795,10126,9669,5072,129,885,144,20,13087,458


In [ ]:
# group 4-13
ctypop.groupby('CTYNAME',)

In [379]:
ctypop.loc[ctypop.AGEGRP==5]

,Unnamed: 0,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,totalWA,totalBA,totalIA,totalAA,totalNA,totalH,totalTOM
5,594,50,53,21,Washington,Franklin,2,5,6584,3445,3139,1882,171,37,101,16,4256,121
24,708,50,53,25,Washington,Grant,2,5,6409,3355,3054,2485,64,66,66,8,3574,146
43,936,50,53,33,Washington,King,2,5,132486,67775,64711,63160,10395,907,29411,1383,18306,8924
62,1278,50,53,45,Washington,Mason,2,5,3295,1881,1414,2237,93,150,84,20,514,197
81,1506,50,53,53,Washington,Pierce,2,5,60391,32973,27418,33501,5244,825,3905,1063,10326,5527
100,1734,50,53,61,Washington,Snohomish,2,5,44780,23114,21666,26842,1935,591,4918,322,7026,3146
119,1791,50,53,63,Washington,Spokane,2,5,34925,17887,17038,26492,998,651,978,276,3396,2134
138,2190,50,53,77,Washington,Yakima,2,5,17013,8658,8355,4705,150,685,143,14,10930,386


In [386]:
cPOP = ctypop.loc[ctypop.AGEGRP >= 4]

In [387]:
cPOP = cPOP.loc[cPOP.AGEGRP < 14]

In [392]:
workage = pd.DataFrame(cPOP.groupby('CTYNAME')['TOT_POP'].sum())

In [399]:
workage.reset_index(inplace=True)

In [400]:
workage.columns

Index(['CTYNAME', 'TOT_POP'], dtype='object')

In [401]:
ageZIP = dict(zip(workage['CTYNAME'], workage['TOT_POP']))
riskrate['working_pop']=riskrate['county'].map(ageZIP).fillna('00')

In [403]:
riskrate[['working_pop','male_pop','fem_pop']] = riskrate[['working_pop','male_pop','fem_pop']].replace(r'\.0$', '', regex=True).astype(int)

In [405]:
riskrate["working_perc"] = riskrate["working_pop"]/riskrate["county_pop"]

In [406]:
riskrate

,Unnamed: 0,zipcode,rate,zip_pop,SQMI,city,county,county_pop,percent_pop,male_pop,fem_pop,wa_pop,ba_pop,ia_pop,aa_pop,na_pop,h_pop,tom_pop,working_pop,working_perc
0,0,99301,9.4,88426,685.57,Pasco,Franklin,97076,0.910895,50042,47034,38409,1973,482,2102,200,52240,1670,61608,0.634637
1,1,98444,8.6,36115,6.88,Tacoma,Pierce,922828,0.039135,463150,459678,596274,66213,10567,63526,15110,109959,61179,611733,0.662890
2,2,98387,7.0,49157,26.79,Spanaway,Pierce,922828,0.053268,463150,459678,596274,66213,10567,63526,15110,109959,61179,611733,0.662890
3,3,98837,6.4,45823,487.02,Moses Lake,Grant,99365,0.461158,50884,48481,52080,931,860,950,140,42682,1722,61130,0.615207
4,4,98404,6.4,36406,7.85,Tacoma,Pierce,922828,0.039450,463150,459678,596274,66213,10567,63526,15110,109959,61179,611733,0.662890
5,5,98391,6.4,56846,40.31,Bonney Lake,Pierce,922828,0.061600,463150,459678,596274,66213,10567,63526,15110,109959,61179,611733,0.662890
6,6,98951,6.4,14412,131.17,Wapato,Yakima,256533,0.056180,128596,127937,106301,2197,9223,2976,211,131130,4495,157286,0.613122
7,7,98003,6.4,50704,12.08,Federal Way,King,2272571,0.022311,1150786,1121785,1293953,152734,13090,456352,17708,230889,107845,1588830,0.699133
8,8,98901,6.2,33463,159.09,Yakima,Yakima,256533,0.130443,128596,127937,106301,2197,9223,2976,211,131130,4495,157286,0.613122
9,9,98223,6.2,45652,336.58,Arlington,Snohomish,829331,0.055047,418871,410460,553800,30510,8630,102433,5028,91517,37413,557494,0.672221


In [407]:
riskrate.to_csv("riskrate.csv")

In [315]:
countypops = ctypop.loc[ctypop.AGEGRP==0]

In [461]:
demo_nhr = pd.read_csv("demographic_lowrisk.csv")

In [462]:
nhr = demo_nhr.loc[demo_nhr.AGEGRP >= 4] 

In [463]:
nhr = nhr.loc[nhr.AGEGRP < 14] 

In [465]:
nhrworkage = pd.DataFrame(nhr.groupby('CTYNAME')['TOT_POP'].sum())

In [467]:
nhr_totpop = demo_nhr.loc[demo_nhr.AGEGRP == 0] 

nhr_totpop

,Unnamed: 0,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,totalWA,totalBA,totalIA,totalAA,totalNA,totalH,totalTOM
0,19,50,53,1,Washington,Adams,2,0,20605,10556,10049,6567,129,98,151,12,13461,187
19,76,50,53,3,Washington,Asotin,2,0,22290,10930,11360,19872,157,332,223,93,1025,588
38,133,50,53,5,Washington,Benton,2,0,207230,104470,102760,142011,2857,1440,6530,377,48649,5366
57,190,50,53,7,Washington,Chelan,2,0,79185,39952,39233,53213,418,544,820,143,22722,1325
76,247,50,53,9,Washington,Clallam,2,0,77280,38287,38993,63101,730,3494,1448,124,5530,2853
95,304,50,53,11,Washington,Clark,2,0,505013,251039,253974,386692,11221,3610,25130,4457,54199,19704
114,361,50,53,13,Washington,Columbia,2,0,3949,1943,2006,3288,32,52,52,85,330,110
133,418,50,53,15,Washington,Cowlitz,2,0,110924,55394,55530,91755,1036,1598,1692,416,10710,3717
152,475,50,53,17,Washington,Douglas,2,0,42998,21945,21053,26846,207,378,403,93,14178,893
171,532,50,53,19,Washington,Ferry,2,0,7178,3674,3504,5204,46,1092,90,23,365,358


In [478]:
nhrworkage["working_pop"] = nhr_totpop["TOT_POP"].values

In [483]:
nhrworkage["working_perc"] =nhrworkage["TOT_POP"]/nhrworkage["working_pop"]

In [486]:
nhrworkage.rename(columns={'TOT_POP':'work_pop','working_pop':'tot_pop'}, inplace=True)

In [488]:
nhrworkage.to_csv("nhr_workingpop.csv")